In [ ]:
# Esse codigo é um exemplo de como criar um ambiente virtual e instalar dependências em Python.
#pip install -r requirements.txt

## Organizando os dados e bibliotecas 

In [30]:
# Bibliotecas necessárias
import pandas as pd
import json
import cvxpy as cp


In [31]:
df = pd.read_excel('Data.xlsx')

In [34]:
# Caminho do arquivo Excel
excel_path = 'Data.xlsx'

# Carregar a planilha
df = pd.read_excel(excel_path)
df.columns = df.columns.str.strip()

# 🔁 ORDENAR antes de numerar
df = df.sort_values('Banco').reset_index(drop=True)

# Gerar número incremental por banco
df['numero'] = df.groupby('Banco').cumcount() + 1
df['DMU'] = df['Banco'] + '_' + df['numero'].astype(str).str.zfill(2)

# Selecionar colunas no formato DEA
df_dea = df[[
    'DMU',
    'Despesa de Intermediação Financeira',
    'Despesa Administrativa e Operacionais',
    'Receita de Intermediação Financeira',
    'Receita com prestação de serviços',
    'Outras receitas operacionais'
]]


In [36]:
# Salvar em CSV
df_dea.to_csv('dados_dea.csv', index=False)

## MODELO 1 - DEA CONSTANTE DE ESCALA MINIMIZANDO AS DESPESAS

In [37]:
# Carrega os dados do CSV
df = pd.read_csv("dados_dea.csv")

In [38]:
# Definir inputs e outputs
inputs = [
    "Despesa de Intermediação Financeira",
    "Despesa Administrativa e Operacionais"
]
outputs = [
    "Receita de Intermediação Financeira",
    "Receita com prestação de serviços",
    "Outras receitas operacionais"
]

In [39]:
# Corrigir: tornar os inputs positivos
df[inputs] = df[inputs].abs()

# (Opcional) Reduzir escala para facilitar o cálculo numérico
df[inputs + outputs] = df[inputs + outputs] / 1e6

In [40]:
# Extrair as matrizes para o modelo DEA
X = df[inputs].values
Y = df[outputs].values
n = X.shape[0]

In [41]:
efficiencies = []

for j in range(n):
    lambdas = cp.Variable(n, nonneg=True)
    theta = cp.Variable()

    constraints = [
        X.T @ lambdas <= theta * X[j],
        Y.T @ lambdas >= Y[j],
        cp.sum(lambdas) == 1
    ]

    problem = cp.Problem(cp.Minimize(theta), constraints)
    problem.solve()

    value = float(theta.value) if theta.value is not None else None
    efficiencies.append(round(value, 4) if value is not None else None)

In [42]:
# Resultados
df_resultado = pd.DataFrame({
    "DMU": df["DMU"],
    "Eficiência DEA (CRS Input-oriented)": efficiencies
})
df_resultado

,DMU,Eficiência DEA (CRS Input-oriented)
0,BB_01,1.0000
1,BB_02,0.9772
2,BB_03,1.0000
3,BB_04,1.0000
4,BB_05,1.0000
5,BB_06,1.0000
6,Itaú_01,0.9199
7,Itaú_02,0.9445
8,Itaú_03,1.0000
9,Itaú_04,0.9297


In [43]:
# Extrair o nome do banco (antes do "_") para nova coluna
df_resultado["Banco"] = df_resultado["DMU"].str.extract(r"(^[A-Za-z]+)")

# Agrupar por banco e tirar a média das eficiências
media_por_banco = df_resultado.groupby("Banco")["Eficiência DEA (CRS Input-oriented)"].mean().reset_index()

# Arredondar para 4 casas decimais
media_por_banco["Eficiência DEA (CRS Input-oriented)"] = media_por_banco["Eficiência DEA (CRS Input-oriented)"].round(4)

media_por_banco

,Banco,Eficiência DEA (CRS Input-oriented)
0,BB,0.9962
1,Ita,0.9271
2,Santander,0.9954
